In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!unzip "/content/drive/My Drive/Data/fc33077e-6-dataset.zip"

In [0]:
#!pip install catboost

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Apr  4 03:29:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8     8W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from hyperopt import hp
import gc
import xgboost as xgb
import catboost as ctb
from sklearn.metrics import mean_absolute_error
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [6]:
%%time
train = pd.read_csv('/content/dataset/train.csv')

CPU times: user 43.8 ms, sys: 13 ms, total: 56.8 ms
Wall time: 59.4 ms


In [0]:
X = train.drop(columns = ['air_pollution_index'])
y = train['air_pollution_index']

In [8]:
X.shape

(33750, 13)

In [0]:
def prepare_df(train):
    org_keys = train.columns.tolist()
    
    train['is_holiday'] = np.where(train['is_holiday'] == 'None', False, True)
    train['date_time'] = pd.to_datetime(train['date_time'])
    
    train.set_index('date_time', inplace = True)
    
    days = ['01d', '03d', '07d', '14d', '30d']
    
    sum_attributes = ['is_holiday', 'rain_p_h', 'snow_p_h']

    for val in sum_attributes:
        for day in days:
            train['num_'+val+'_'+day] = train.rolling(day)[val].sum()
            
    weather_types = train['weather_type'].unique().tolist()
    for weather in weather_types:
        train[weather] = np.where(train['weather_type'] == weather, 1, 0)
    
    train.drop(columns = 'weather_type', inplace = True)
    
    avg_attributes = ['humidity', 'wind_speed', 'wind_direction', 'visibility_in_miles', 'dew_point', 'temperature', 
                     'rain_p_h', 'snow_p_h', 'clouds_all', 'traffic_volume'] + weather_types

    for val in avg_attributes:
        for day in days:
            train['avg_'+val+'_'+day] = train.rolling(day)[val].mean()
            
    for column in train.columns:
        if 'avg' in column:
            train[column+'_diff'] = train[column[4:-4]] - train[column]
            
    return train.reset_index()

In [10]:
%%time
X = prepare_df(X.copy())

CPU times: user 635 ms, sys: 63.1 ms, total: 698 ms
Wall time: 699 ms


In [11]:
X.shape

(33750, 248)

In [0]:
X['date_time'] = X['date_time'].dt.hour

In [13]:
%%time
selector = VarianceThreshold(0.1)
selector.fit(X)

CPU times: user 420 ms, sys: 95 ms, total: 515 ms
Wall time: 515 ms


In [0]:
sel = X.columns[selector.get_support(indices=True)]

In [0]:
X = X[sel]

In [0]:
rfe = RFE(xgb.XGBRFRegressor(ojective = 'reg:squarederror', tree_method = 'gpu_hist'), step = 1, verbose = 2)

In [17]:
%%time
rfe.fit(X, y)

Fitting estimator with 121 features.
[03:29:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 120 features.
[03:29:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 119 features.
[03:29:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 118 features.
[03:29:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 117 features.
[03:29:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimator with 116 features.
[03:29:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting estimato

RFE(estimator=XGBRFRegressor(base_score=0.5, colsample_bylevel=1,
                             colsample_bynode=0.8, colsample_bytree=1, gamma=0,
                             learning_rate=1, max_delta_step=0, max_depth=3,
                             min_child_weight=1, missing=None, n_estimators=100,
                             n_jobs=1, nthread=None, objective='reg:linear',
                             ojective='reg:squarederror', random_state=0,
                             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                             seed=None, silent=None, subsample=0.8,
                             tree_method='gpu_hist', verbosity=1),
    n_features_to_select=None, step=1, verbose=2)

In [0]:
sel = X.columns[rfe.get_support(indices=True)]

In [0]:
X = X[sel]

In [0]:
X_train, X_test = X[:27000], X[27000:]
y_train, y_test = y[:27000], y[27000:]

In [0]:
def xgbrf_score(params):
    estimator = xgb.XGBRFRegressor(**params)
    estimator.fit(X_train, y_train)
    predictions = estimator.predict(X_test)
    loss = mean_absolute_error(y_test, np.array(predictions))
    del estimator, predictions
    gc.collect()
    return {'loss': loss, 'status': STATUS_OK}

In [0]:
def xgbrf_optimize(evals, trials, optimizer=tpe.suggest, random_state=0):
    space = {
        'n_estimators': hp.choice('n_estimators', np.arange(200, 600, dtype=int)),
        'eta': hp.quniform('eta', 0.025, 0.25, 0.025), # A problem with max_depth casted to float instead of int with the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
        'subsample': hp.quniform('subsample', 0.7, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.7, 1, 0.05),
        'alpha' :  hp.quniform('alpha', 0, 10, 1),
        'lambda': hp.quniform('lambda', 1, 2, 0.1),
        #'nthread': cores,
        'objective': 'reg:squarederror',
        'tree_method': 'gpu_hist',
        #'n_jobs': -1,
        'seed': random_state
    }
    best = fmin(xgbrf_score, space, algo=tpe.suggest, max_evals=evals, trials = trials)
    return best

In [0]:
%%time
trials = Trials()
n= 1000
xgbrf_best_param = xgbrf_optimize(evals = n,
                      optimizer=tpe.suggest,
                      trials = trials)

  0%|          | 4/1000 [00:42<2:19:07,  8.38s/it, best loss: 72.59559394892939]

In [0]:
xgbrf_best_param['objective'] = 'reg:squarederror'
xgbrf_best_param['tree_method'] = 'gpu_hist'
xgbrf_best_param['max_depth'] = max(xgbrf_best_param['max_depth'], 6)

In [0]:
xgbrf_estimator = xgb.XGBRFRegressor(**xgbrf_best_param)

In [0]:
%%time
xgbrf_estimator.fit(X_train, y_train)

In [0]:
xgbrf_score = mean_absolute_error(y_test, np.array(xgbrf_estimator.score(X_test)))
print(xgbrf_score)

In [0]:
def ctb_score(params):
    estimator = ctb.CatBoostRegressor(**params)
    estimator.fit(X_train, y_train)
    predictions = estimator.predict(X_test)
    loss = mean_absolute_error(y_test, np.array(predictions))
    del estimator, predictions
    gc.collect()
    return {'loss': loss, 'status': STATUS_OK}

In [0]:
def ctb_optimize(evals, trials, optimizer=tpe.suggest, random_state=0):
    space = {
            'depth': hp.choice('depth', np.arange(3, 12, dtype=int)),
            'max_ctr_complexity': hp.choice('max_ctr_complexity', np.arange(1, 16, dtype=int)),
            'border_count': hp.choice('border_count', [64, 128, 255]),
            'ctr_description': hp.choice('ctr_description', ['FeatureFreq']),
            'learning_rate': hp.loguniform('learning_rate', -5, 0),
            #'random_strength': hp.choice('random_strength', np.arange(3, 50, dtype=int)),
            'one_hot_max_size': hp.choice('one_hot_max_size', np.arange(0, 25, dtype=int)),
            'l2_leaf_reg': hp.loguniform('l2_leaf_reg', 0, np.log(10)),
            'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),
            #'used_ram_limit': hp.choice('used_ram_limit', [100000000000]),
            'task_type': 'GPU',
            'verbose': 0
        }
    best = fmin(ctb_score, space, algo=tpe.suggest, max_evals=evals, trials = trials)
    return best

In [0]:
%%time
trials = Trials()
n= 1000
ctb_best_param = ctb_optimize(evals = n,
                      optimizer=tpe.suggest,
                      trials = trials)

In [0]:
ctb_best_param['tree_method'] = 'gpu_hist'
ctb_best_param['max_depth'] = max(ctb_best_param['max_depth'], 3)
ctb_best_param['verbose'] = 0

In [0]:
ctb_estimator = ctb.CatBoostRegressor(**ctb_best_param)

In [0]:
%%time
fit(X_train, y_train)

In [0]:
ctb_score = mean_absolute_error(y_test, np.array(ctb_estimator.predict(X_test)))
print(ctb_score)

In [0]:
score = xgbrf_score + ctb_score

In [0]:
test = pd.read_csv('/content/dataset/test.csv')

In [0]:
%%time
X_sub = prepare_df(test.copy())

In [0]:
X_sub['date_time'] = X_sub['date_time'].dt.hour

In [0]:
X_sub = X_sub[sel]

In [0]:
test['air_pollution_index'] = xgbrf_estimator.predict(X_sub)*score/xgbrf_score + ctb_estimator.predict(X_sub)*score/ctb_score

In [0]:
test = test.reset_index()

In [0]:
test.to_csv('sub5.csv', columns = ['date_time', 'air_pollution_index'], index = False)